# Search for accusing tweets

In this notebook we will search for accusing tweets. We mean by this replies of this kind:

- "you are a racist"
- "You are a misoginist"
- "you hate black people" 

and so on

In [88]:
%load_ext autoreload
%autoreload 2
from mongoengine import connect
from tweepyrate import create_apps
from hate_collector.models import Tweet

client = connect("hatespeech")
db = client["hatespeech"]

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Let's look for tweets containing "sos machista"

In [89]:
accusing_tweets = Tweet.objects(__raw__={
    "$text": {"$search": "machista misogino"},
    "in_reply_to_status_id": {"$ne": None},
    "$expr": {"$lt": [{"$strLenCP": "$text"}, 100]},
})

print(len(accusing_tweets))
for tweet in accusing_tweets[:20]:
    print(f"-- {tweet.text}")

3385
-- @ExpositoCOPE Machista,machista y machista,porque no puede ella trabajar igualnque el?
-- @mujer_alienada Machista por que si, machista por que no y machista por si acaso xD
-- @jmarialg más machistas que nunca, es la contrarreacción machista
-- @ExpositoCOPE Machista, a eso mi madre lo llama machista.
-- @FemiCadiz Machista coño,VIOLENCIA MACHISTA
-- @JonHeguier Y más machista!!!!! Te faltó decir que es machista en extremo.
-- ¡MACHISTA!
-- machista*
-- @Dayana21586226 es machista, o acaso los machistas son buenas personas?
-- Y sobre todo es machista.
-- @j7dios2018 Maldito machista, ojala se mueran todos los machistas
-- @noticiasrevista Dame la nota más machista q tenga... No, no tan machista
-- @katelights No hay nada mas parecido a un machista de derechas que un machista de izquierda.
-- @RadichetConv El comentario mas machista que tengas @RadichetConv 
No, no tan machista
-- @ReasonToLive666 @santisolari @Milly427 Mucho tenemos un chip machista, tengo un chip machista 🤦


How many of these replies have their original tweet in db?


In [90]:
upstream_ids = set(tweet.in_reply_to_status_id for tweet in accusing_tweets)

print(f"There are {len(upstream_ids)} tweets to be retrieved")

upstream_tweets = Tweet.objects(id__in=upstream_ids)
print(f"There are {upstream_tweets.count()} in DB")

There are 2971 tweets to be retrieved
There are 2877 in DB


I will look only for those which don't have any urls inside


In [91]:

upstream_tweets = Tweet.objects(__raw__={
    # Let me check all the tweets
    "_id": {"$in": list(upstream_ids)},
    "entities.urls": {"$size": 0},
    "entities.media": {"$exists": False},
    "in_reply_to_user_id": None,
})

print(f"There are {upstream_tweets.count()} without urls and media")

for tweet in upstream_tweets[:100]:
    print(f"\n-- {tweet.text}")

There are 573 without urls and media

-- FELIZ ANO NOVO POVO

-- "Sex education" puede ser mejor o peor que otras series, peeo por lo menos tiene un personaje gay NO HETERONORMATIVO, que eso es algo que nunca pasa...

-- Cuando a algún hombre que conozcáis, o a vosotros mismos os caiga una denuncia falsa muy rica por parte de una mujer sin escrúpulos, le dais las gracias a los millones de votantes del PSOE. Yo he intentado cambiar eso con mi voto. Quería igualdad para todas las personas. Y quiero

-- La prostitución lleva "debatiéndose" desde los 70. Ahora ya es hora de debatir cómo abolirla y dejar de defender posturas que favorecen a los hombres y al sistema capitalista neoliberal.

-- No quiero bancos con los colores del arcoiris.
No quiero la bandera LGTBI en sitios públicos.
No quiero desfile del Orgullo LGTBI.
No quiero que se casen.
No quiero que adopten.
No quiero que muestren afecto en público. 
Apoyo terapias de conversión.

Pero no soy homófobo 🤷🏽‍♂️

-- Y avisenle a wendi q

# Removed tweets

In [103]:
from hate_collector.models import APIError


removed_tweets_errors = APIError.objects(api_code=422)

removed_tweets = Tweet.objects(id__in=[e.tweet_id for e in removed_tweets_errors])

In [105]:
 APIError.objects(api_code=422).count()

8

In [104]:
for tw in removed_tweets:
    print(f"-- {tw.text}")

In [54]:
tweet = Tweet.objects.search_text('"¿Y tú qué insulto progre eres?"')[0]

In [57]:
vars(tweet)

{'_cls': 'Tweet',
 '_dynamic_lock': False,
 '_fields_ordered': ('id',
  'text',
  'created_at',
  'id_str',
  'full_text',
  'truncated',
  'display_text_range',
  'entities',
  'extended_entities',
  'source',
  'in_reply_to_status_id',
  'in_reply_to_status_id_str',
  'in_reply_to_user_id',
  'in_reply_to_user_id_str',
  'in_reply_to_screen_name',
  'user',
  'geo',
  'coordinates',
  'place',
  'contributors',
  'is_quote_status',
  'retweet_count',
  'favorite_count',
  'favorited',
  'retweeted',
  'possibly_sensitive',
  'possibly_sensitive_appealable',
  'lang'),
 'id_str': '1214550859633381388',
 'full_text': '¿Y tú qué insulto progre eres? https://t.co/HKlPe0gwt6',
 'truncated': False,
 'display_text_range': [0, 30],
 'entities': {'hashtags': [],
  'symbols': [],
  'user_mentions': [],
  'urls': [],
  'media': [{'id': 1214550850712014850,
    'id_str': '1214550850712014850',
    'indices': [31, 54],
    'media_url': 'http://pbs.twimg.com/media/ENrzkFJWkAI05Qz.jpg',
    'media_